In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as tgrad

import os
import time
import utils
import networks
import numpy as np
import matplotlib.pyplot as plt

import importlib

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
# torch.set_default_tensor_type(torch.DoubleTensor)
print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

True
cuda


# Data Sampling
Here in our case, the system is European Call Option PDE and the physical information about the system consists of Boundary Value conditions, final Value conditions and the PDE itself.

In [3]:
K = 40
r = 0.05
sigma = 0.25
T = 1
S_range = [0, 130]
t_range = [0, T]
gs = lambda x: np.fmax(x-K, 0)

# Build Neural Network

In [4]:
fnn = networks.FeedforwardNeuralNetwork(2, 50, 1, 3)
fnn.cuda()

FeedforwardNeuralNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1-2): 2 x Linear(in_features=50, out_features=50, bias=True)
  )
  (output): Linear(in_features=50, out_features=1, bias=True)
  (relu): ReLU()
)

In [5]:
n_epochs = 60000
lossFunction = nn.MSELoss()
optimizer = optim.Adam(fnn.parameters(), lr=0.00002)

samples = {"pde": 50000, "bc":25000, "fc":25000}

# Modelling

- For each iteration in the training loop, we are sampling data for the three physical conditions of the PDE.
- Then we are calculating the loss three times on the same model, accumulating them into a combined objective function to be minimised for the Neural Network.
- The first loss is the differential equation loss. Here we are trying to minimise the PDE by calculating gradients and forming the PDE itself.
- The remaining losses are calculated for boundary value and initial value conditions for the PDE.
- Mean Squared Error loss function `nn.MSELoss()` is chosen as the criterion to be minimised and 
- Adam optimizer `nn.optim.Adam(lr=3e-5)` with a learning rate of 0.00003 is chosen for performing the weight updates.

In [6]:
loss_hist = []
start_time = time.time()

for epoch in range(n_epochs):
    
    f_st_train, f_v_train, bc_st_train, bc_v_train, n_st_train, n_v_train = \
    utils.trainingData3(K, 
                       r, 
                       sigma, 
                       T, 
                       S_range[-1], 
                       S_range, 
                       t_range, 
                       gs, 
                       samples['bc'], 
                       samples['fc'], 
                       samples['pde'], 
                       RNG_key=123)
    
    # save training data points to tensor and send to device
    f_st_train = torch.from_numpy(f_st_train).float().requires_grad_().to(device)
    f_v_train = torch.from_numpy(f_v_train).float().to(device)
    
    n_st_train = torch.from_numpy(n_st_train).float().requires_grad_().to(device)
    n_v_train = torch.from_numpy(n_v_train).float().to(device)
    
    bc_st_train = torch.from_numpy(bc_st_train).float().to(device)
    bc_v_train = torch.from_numpy(bc_v_train).float().to(device)
    
    
    # PDE Round
    y1_hat = fnn(n_st_train)
    
    grads = tgrad.grad(y1_hat, n_st_train, grad_outputs=torch.ones(y1_hat.shape).cuda(), retain_graph=True, create_graph=True, only_inputs=True)[0]
    # print(grads)
    dVdt, dVdS = grads[:, 0].view(-1, 1), grads[:, 1].view(-1, 1)
    grads2nd = tgrad.grad(dVdS, n_st_train, grad_outputs=torch.ones(dVdS.shape).cuda(), create_graph=True, only_inputs=True)[0]
    # print(grads2nd)
    d2VdS2 = grads2nd[:, 1].view(-1, 1)
    S1 = n_st_train[:, 1].view(-1, 1)
    pde_loss = lossFunction(-dVdt, 0.5*((sigma*S1)**2)*d2VdS2 + r*S1*dVdS - r*y1_hat)
    
    
    # BC Round
    y21_hat = fnn(bc_st_train)
    bc_loss = lossFunction(bc_v_train, y21_hat)
    
    # final condition loss
    fc_hat = fnn(f_st_train)
    fc_loss = lossFunction(f_v_train, fc_hat)
    
    
    # Backpropagation and Update
    optimizer.zero_grad()
    combined_loss = pde_loss + bc_loss + fc_loss
    combined_loss.backward()
    optimizer.step()
    
    loss_hist.append(combined_loss.item())
    if epoch % 500 == 0:
        print(f'{epoch}/{n_epochs} PDE Loss: {pde_loss.item():.5f}, BC Loss: {bc_loss.item():.6f}, fc loss: {fc_loss.item(): 5f}, total loss: {combined_loss.item():5f}, minimum loss: {min(loss_hist):.5f}')

end_time = time.time()
print('run time:', end_time - start_time)

0/60000 PDE Loss: 0.00012, BC Loss: 4275.059570, fc loss:  1942.970947, total loss: 6218.030273, minimum loss: 6218.03027
500/60000 PDE Loss: 0.00023, BC Loss: 3161.380371, fc loss:  1364.743652, total loss: 4526.124023, minimum loss: 4526.12402
1000/60000 PDE Loss: 0.00584, BC Loss: 1825.971802, fc loss:  715.874756, total loss: 2541.852539, minimum loss: 2541.85254
1500/60000 PDE Loss: 0.04883, BC Loss: 572.268738, fc loss:  214.140869, total loss: 786.458435, minimum loss: 786.45844
2000/60000 PDE Loss: 0.30044, BC Loss: 94.363739, fc loss:  139.145569, total loss: 233.809753, minimum loss: 233.80975
2500/60000 PDE Loss: 1.12589, BC Loss: 30.544010, fc loss:  155.012863, total loss: 186.682770, minimum loss: 186.68277
3000/60000 PDE Loss: 2.42080, BC Loss: 23.008890, fc loss:  147.799713, total loss: 173.229401, minimum loss: 173.22940
3500/60000 PDE Loss: 3.98866, BC Loss: 20.324154, fc loss:  137.362778, total loss: 161.675583, minimum loss: 161.67558
4000/60000 PDE Loss: 6.18169,